In [2]:
!ls "../dataset"
!ls "../model"

neg.txt  unknown.pkl	 wn_neg_tok.pkl  wn_pos_int.npy  wn_pos.txt
pos.txt  wn_neg_int.npy  wn_neg.txt	 wn_pos_tok.pkl  wongnai
thai2vec  vec.npy


In [5]:
%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

import glob, re, time, sys
import itertools

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, MaxPooling1D, Dropout, Flatten, Bidirectional, GRU, Embedding
from keras import optimizers
from keras import utils

Using TensorFlow backend.


In [8]:
learning_rate = 0.0001 # 1e-4
num_steps = 10
batch_size = 64
epochs = 10

data_dim = 300
input_len = 100
num_classes = 2

# pos = np.load('pos.npy')
# neg = np.load('neg.npy')

pos = np.load('../dataset/wn_pos_int.npy')
neg = np.load('../dataset/wn_neg_int.npy')

word_vec = np.load('../model/vec.npy')

x = np.concatenate((pos,neg), axis = 0)
y = np.concatenate(
	(
		np.full((pos.shape[0], 2), [0,1]),
		np.full((neg.shape[0], 2), [1,0])
	), axis = 0
)
# x shape is (num_data, 100, 300) 100 is sentence length, 300 is vector dim
# y shape is (num_data, 2)
# y result are [1, 0] if it's a neg else it's [0, 1]
x_train, x_val, y_train, y_val =  train_test_split(x,y, test_size=0.2)

dropout = 0.25

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_val shape:', x_val.shape)
print('y_val shape:', y_val.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'test samples')

kernel_size = 5
pool_size = 4

x_train shape: (7187, 600)
y_train shape: (7187, 2)
x_val shape: (1797, 600)
y_val shape: (1797, 2)
7187 train samples
1797 test samples


In [9]:
model = Sequential()

# model.add(Conv1D(64, kernel_size, activation='relu', input_shape=(input_len, data_dim)))
# model.add(MaxPooling1D(pool_size=pool_size))
model.add(Embedding(len(word_vec), 300, input_length=600, weights=[word_vec], trainable=False))
model.add(Bidirectional(GRU(128, return_sequences=True)))
model.add(Bidirectional(GRU(64)))
model.add(Dense(32, activation='tanh'))
model.add(Dense(2, activation='softmax'))

adam = optimizers.Adam(lr=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))
print(model.summary())

model_name = 'model.h5'
# if os.path.exists(model_name):
# 	model.load_weights(model_name)
# 	print("Loaded model from disk")
# else:
model.save_weights(model_name)
print("Saved model to disk")

rand = np.random.choice(len(x_val), 10, replace=False)
for i in rand:
	pre_x, pre_y = x_val[i], y_val[i]
	pre_x = pre_x.reshape(1, 600)
	res_y = model.predict_classes(pre_x, batch_size=1)
	print('Prediction: {}'.format(res_y[0]))
	print('Result: {}'.format(str(np.where(pre_y==1.)[0])[1]))

score = model.evaluate(x_val, y_val, batch_size=batch_size)
print("{}: {}".format(model.metrics_names[0], score[0]))
print("{}: {}%".format(model.metrics_names[1], score[1]*100))

Train on 7187 samples, validate on 1797 samples
Epoch 1/10
7187/7187 [==============================] - 185s 26ms/step - loss: 0.5761 - acc: 0.7409 - val_loss: 0.5593 - val_acc: 0.7496
Epoch 2/10
7187/7187 [==============================] - 182s 25ms/step - loss: 0.5562 - acc: 0.7507 - val_loss: 0.5721 - val_acc: 0.7496
Epoch 3/10
7187/7187 [==============================] - 182s 25ms/step - loss: 0.5514 - acc: 0.7507 - val_loss: 0.5535 - val_acc: 0.7496
Epoch 4/10
7187/7187 [==============================] - 182s 25ms/step - loss: 0.5463 - acc: 0.7502 - val_loss: 0.5533 - val_acc: 0.7496
Epoch 5/10
7187/7187 [==============================] - 182s 25ms/step - loss: 0.5409 - acc: 0.7512 - val_loss: 0.5468 - val_acc: 0.7485
Epoch 6/10
7187/7187 [==============================] - 182s 25ms/step - loss: 0.5339 - acc: 0.7515 - val_loss: 0.5424 - val_acc: 0.7490
Epoch 7/10
7187/7187 [==============================] - 182s 25ms/step - loss: 0.5227 - acc: 0.7541 - val_loss: 0.5290 - val_acc: 

In [10]:
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
1